# Feature Extraction

### Bag of words

In [3]:
docs = ["SUPERB, I AM IN LOVE IN THIS PHONE", "I hate this phone"]
words = list(set([word for doc in docs for word in doc.lower().split()]))
vectors = []
for doc in docs:
    vectors.append([1 if word in doc.lower().split() else 0 for word in words])
print("vocabulary: ", words)   
print("vectors: ", vectors)

vocabulary:  ['i', 'superb,', 'in', 'am', 'hate', 'love', 'phone', 'this']
vectors:  [[1, 1, 1, 1, 0, 1, 1, 1], [1, 0, 0, 0, 1, 0, 1, 1]]


### Word counts with CountVectorizer(scikitlearn)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
# list of documents
docs = ['SUPERB, I AM IN LOVE IN THIS PHONE', 'I hate this phone']
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(docs)
print('vocabulary: ', vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(docs)
# summarize encoded vector
print('shape: ', vector.shape)
print('vectors: ', vector.toarray())

vocabulary:  {'superb': 5, 'am': 0, 'in': 2, 'love': 3, 'this': 6, 'phone': 4, 'hate': 1}
shape:  (2, 7)
vectors:  [[1 0 2 1 1 1 1]
 [0 1 0 0 1 0 1]]


### TF-IDF Vectorizer

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of documents
docs = ["SUPERB, I AM IN LOVE IN THIS PHONE", 
        "I hate this phone"]
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(docs)
# summarize
print('vocabulary: ', vectorizer.vocabulary_)
print('idfs: ', vectorizer.idf_)
# encode document
vector = vectorizer.transform([docs[0]])
# summarize encoded vector
print('vectors: ', vector.toarray())

vocabulary:  {'superb': 5, 'am': 0, 'in': 2, 'love': 3, 'this': 6, 'phone': 4, 'hate': 1}
idfs:  [1.40546511 1.40546511 1.40546511 1.40546511 1.         1.40546511
 1.        ]
vectors:  [[0.35327777 0.         0.70655553 0.35327777 0.25136004 0.35327777
  0.25136004]]


### Sentiment Analysis

In [9]:
import pandas as pd
# train Data
trainData = pd.read_csv("https://raw.githubusercontent.com/Vasistareddy/sentiment_analysis/master/data/train.csv")
# test Data
testData = pd.read_csv("https://raw.githubusercontent.com/Vasistareddy/sentiment_analysis/master/data/test.csv")

In [10]:
trainData.sample(frac=1).head(5) 
# shuffle the df and pick first 5

,Content,Label
287,anna and the king strides onto the screen in f...,pos
530,"in the grand scheme of mel gibson movies , pay...",pos
1526,the corruptor is a big silly mess of an action...,neg
1204,hello kids . \ntoday the movie studios want to...,neg
885,jamaica is a hot vacation spot . \nthe exchang...,pos


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['Content'])
test_vectors = vectorizer.transform(testData['Content'])

### Creating a linear SVM model

In [13]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, trainData['Label'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['Label'], prediction_linear, output_dict=True)
print('positive: ', report['pos'])
print('negative: ', report['neg'])

Training time: 5.296200s; Prediction time: 0.516672s
positive:  {'precision': 0.9191919191919192, 'recall': 0.91, 'f1-score': 0.9145728643216081, 'support': 100}
negative:  {'precision': 0.9108910891089109, 'recall': 0.92, 'f1-score': 0.9154228855721394, 'support': 100}


In [14]:
review = """SUPERB, I AM IN LOVE IN THIS PHONE"""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

['pos']


In [15]:
review = """Do not purchase this product. My cell phone blast when I switched the charger"""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

['neg']


In [16]:
review = """I received defective piece display is not working properly"""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

['neg']


In [17]:
review = """It's not even 5 days since i purchased this product.
I would say this a specially blended worst Phone in all formats.
ISSUE 1:
Have you ever heard of phone which gets drained even in standby mode during night?
Kindly please see the screenshot if you want to believe my statement.
My phone was in full charge at night 10:07 PM . I took this screenshot and went to sleep.
Then I woke up at morning and 6:35 AM and battery got drained by 56% in just standby condition.
If this is the case consider how many hours it will work, during day time.
It's not even 5 hours the battery is able to withstand.
ISSUE 2:
Apart from the battery, the next issue is the heating issue .I purchased a iron box recently from Bajaj in this sale.
But I realized this phone acts a very good Iron box than the Bajaj Iron box. I am using only my headphones to get connected in the call. I am not sure when this phone is will get busted due to this heating issue. It is definitely a challenge to hold this phone for even 1 minute. The heat that the phone is causing will definitely burn your hands and for man if you keep this phone in your pant pocket easily this will lead to infertility for you. Kindly please be aware about that.
Issue 3:
Even some unknown brands has a better touch sensitivity. The touch sensitivity is pathetic, if perform some operation it will easily take 1-2 minutes for the phone to response.
For your kind information my system has 73% of Memory free and the RAM is also 56% free.
Kindly please make this Review famous and lets make everyone aware of this issue with this phone.
Let's save people from buying this phone. There are people who don't even know what to do if this issue happens after 10 days from the date of purchase. So I feel at least this review will help people from purchasing this product in mere future."""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

['neg']


### Pickle the model

In [19]:
import pickle
# pickling the vectorizer
pickle.dump(vectorizer, open('vectorizer.sav', 'wb'))
# pickling the model
pickle.dump(classifier_linear, open('classifier.sav', 'wb'))